<a href="https://colab.research.google.com/github/rostro36/BERT-test/blob/master/SQuAD_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://hackernoon.com/nlp-tutorial-creating-question-answering-system-using-bert-squad-on-colab-tpu-1utp3352

Unfortunately, the tutorial is written for Tensorflow 1.x

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#Download Bert
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [3]:
cd bert

/content/bert


In [4]:
#download pre-trained model
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
# Unzip the pretrained model
!unzip uncased_L-24_H-1024_A-16.zip

--2020-04-29 12:37:48--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G   213MB/s    in 5.7s    

2020-04-29 12:37:54 (210 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]

Archive:  uncased_L-24_H-1024_A-16.zip
   creating: uncased_L-24_H-1024_A-16/
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: uncased_L-24_H-1024_A-16/bert_config.json  


In [5]:
#Download the SQuAD 2.0 train and dev dataset
#SQuAD 2.0 also has unanswerable questions
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-04-29 12:38:12--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  57.9MB/s    in 0.7s    

2020-04-29 12:38:12 (57.9 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-04-29 12:38:13--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

In [6]:
import os
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

TPU address is =>  grpc://10.77.20.210:8470


In [7]:
import pprint
import tensorflow as tf
from google.colab import auth
import json
auth.authenticate_user()


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 7162335457686223441),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8168675651053508411),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15031940250022392852),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4854495245093273311),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 16270844644830364073),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13003041

In [9]:
with tf.compat.v1.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())
  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)


TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 7162335457686223441),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8168675651053508411),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15031940250022392852),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4854495245093273311),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 16270844644830364073),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 130030412906759834),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 10158826103338310395),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2147324194681014612),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 15614630745400248087),
 _DeviceAttributes(/job:tpu_wor

In [10]:
BUCKET = 'squad_test' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'bert_output' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://squad_test/bert_output *****


In [11]:
!gsutil mv /content/bert/uncased_L-24_H-1024_A-16 $BUCKET_NAME


Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta [Content-Type=application/octet-stream]...
Removing file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta...
Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_config.json [Content-Type=application/json]...
Removing file:///content/bert/uncased_L-24_H-1024_A-16/bert_config.json...
Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Removing file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index...
Copying file:///content/bert/uncased_L-24_H-1024_A-16/vocab.txt [Content-Type=text/plain]...
Removing file:///content/bert/uncased_L-24_H-1024_A-16/vocab.txt...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying f

# !!! Change tpu_name to the correct address from cell [6]

In [12]:
!python run_squad.py \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_model.ckpt \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --use_tpu=True \
  --tpu_name=grpc://10.77.20.210:8470 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --version_2_with_negative=True \
  --output_dir=$OUTPUT_DIR

Streaming output truncated to the last 5000 lines.
I0429 13:52:03.994264 140654771337088 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0429 13:52:04.010974 140654771337088 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0429 13:52:04.011250 140654771337088 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0429 13:52:04.028598 140654771337088 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0429 13:52:04.028838 140654771337088 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0429 13:52:04.046253 140654771337088 tpu_estimator.py:600] Enqueue next (1) batch(es) of data

In [0]:
!touch input_file.json


In [14]:
%%writefile input_file.json
{
    "version": "v2.0",
    "data": [
        {
            "title": "your_title",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "where did Pirmin Schwegler start?",
                            "id": "56ddde6b9a695914005b9628",
                            "is_impossible": ""
                        },
                        {
                            "question": "does he have siblings?",
                            "id": "56ddde6b9a695914005b9629",
                            "is_impossible": ""
                        },
                        {
                            "question": "was he ever sick?",
                            "id": "56ddde6b9a695914005b962a",
                            "is_impossible": ""
                        }
                    ],
                    "context": "Schwegler began his career with FC Luzern. In summer 2005, he joined Young Boys. After just one year with Young Boys he signed with Bayer Leverkusen. On 18 July 2009, after three years with the Werksclub, Schwegler signed with Eintracht Frankfurt. Schwegler, wearing the captain's armband for three years at Frankfurt, left the club in the 2014 summer transfer to join fellow Bundesliga side TSG Hoffenheim on a three-year deal. He signed for Australian club Western Sydney Wanderers, on a one year deal, for their upcoming 2019/2020 season. He is the brother of Christian Schwegler. When he was 16-months-old, he was diagnosed with leukaemia. He was initially only given a 10% chance of survival. Doctors at the University of Bern Hospital's children's ward, led by Dr. Annette Ridolfi Luthy, fought to save him. The chemotherapy was successful but Schwegler had to go back to the clinic for regular check-ups throughout his childhood."                
                 }
            ]
        }
    ]
}

Overwriting input_file.json


In [15]:
!python run_squad.py \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=$OUTPUT_DIR/model.ckpt-10859 \
  --do_train=False \
  --max_query_length=30  \
  --do_predict=True \
  --predict_file=input_file.json \
  --predict_batch_size=8 \
  --n_best_size=3 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=output/




W0429 13:54:08.977285 140665898432384 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0429 13:54:08.977519 140665898432384 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0429 13:54:08.977706 140665898432384 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0429 13:54:10.515959 140665898432384 module_wrapper.py:139] From run_squad.py:1133: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related op

In [16]:
!cat output/nbest_predictions.json

{
    "56ddde6b9a695914005b9628": [
        {
            "text": "FC Luzern",
            "probability": 0.9987392370918314,
            "start_logit": 9.178156852722168,
            "end_logit": 8.711442947387695
        },
        {
            "text": "FC Luzern.",
            "probability": 0.0010592603398322366,
            "start_logit": 9.178156852722168,
            "end_logit": 1.8625200986862183
        },
        {
            "text": "with FC Luzern",
            "probability": 0.0002015025683363416,
            "start_logit": 0.6697099804878235,
            "end_logit": 8.711442947387695
        }
    ],
    "56ddde6b9a695914005b9629": [
        {
            "text": "He is the brother of Christian Schwegler",
            "probability": 0.6654140112018887,
            "start_logit": 5.13279390335083,
            "end_logit": 2.2899773120880127
        },
        {
            "text": "He is the brother of Christian Schwegler.",
            "probability": 0.184682210741922

We can see, that BERT sucessfully answered my (simple, but not trivial) questions.<br>
The experiment was successful.